# Web scraping using Python and PhantomJS

This is a starter kit for developing web scrapers using Python and PhantomJS.


In [1]:

import platform
import time
# from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC


# Using the right PhantomJS for the corresponding OS
if platform.system() == "Windows":
    PHANTOMJS_EXE = "./PhantomJS/phantomjs.exe"
else:
    PHANTOMJS_EXE = "./PhantomJS/phantomjs"

USER_AGENT = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) " + \
             "AppleWebKit/537.36 (KHTML, like Gecko) " + \
             "Chrome/55.0.2883.87 Safari/537.36"

URL_LOGIN = "http://datalogger.capnuocnhabe.vn"
# URL_LOGIN = "http://113.161.69.85:1802/Login.aspx"
ACCOUNT = {'id': "sawaco", 'pw': "123456"}
URL_LOGGER = "http://113.161.69.85:1802/Consumer/Logger/Daily_Monthly.aspx"
START_DATE = '01/09/2017'
END_DATE   = '05/09/2017'
XNTD = ["39 Ben Van Don",
        "65 BEN VAN DON",
        # "90 BEN VAN DON",
        # "BEN VAN DON_NGUYEN KHOAI",
        # "CALMETTE - BEN VAN DON",
        # "CAU HIEP PHUOC",
        # "CAU RACH ONG 1",
        # "Cau Rach Ong 2",
        # "CAU TAC BEN RO",
        # "CTY PT CN TAN THUAN",
        # "D1000 Huynh Tan Phat",
        # "D300 Cau Ba Chiem",
        # "D600 Cau Ong Lon",
        # "D600 NVLinh_NHTho",
        # "KCN HIEP PHUOC",
        # "KCX TThuan",
        # "LONG THOI - NHON DUC",
        # "NGUYEN BINH",
        # "NHTho_PHLau",
        # "NVLinh_NLBang",
        "VUON UOM BOO",
        # "Vuon uom Tan Thuan",
        "D1000 Huynh Tan Phat (ÐC)",
        # "D300 Cau Ba Chiem (ÐC)",
        # "D600 NVLinh_NHTho (ÐC)",
        # "NHTho_PHLau (ÐC)",
        # "NVLinh_NLBang (ÐC)",
        # "Vuon uom BOO (ÐC)",
        # "Vuon uom Tan Thuan (ÐC)",
]

XNTD = ["102 LE QUOC HUNG"]


def site_log_in(driver, account):
    wait = WebDriverWait(driver, 10)
    wait.until(EC.presence_of_element_located((By.ID, "ContentPlaceHolder1_ucLogin1_Login1_LoginButton")))

    # METHOD 1
    # --------
    driver.find_element_by_xpath("//input[@id='ContentPlaceHolder1_ucLogin1_Login1_UserName']").send_keys(account['id'])
    driver.find_element_by_xpath("//input[@id='ContentPlaceHolder1_ucLogin1_Login1_Password']").send_keys(account['pw'])
    driver.find_element_by_xpath("//input[@id='ContentPlaceHolder1_ucLogin1_Login1_LoginButton']").click()

    # METHOD 2
    # --------
    # from selenium.webdriver.common.keys import Keys
    # from selenium.webdriver import ActionChains
    # username_field = driver.find_element_by_xpath("//input[@id='ContentPlaceHolder1_ucLogin1_Login1_UserName']")
    # password_field = driver.find_element_by_xpath("//input[@id='ContentPlaceHolder1_ucLogin1_Login1_Password']")
    # actions = ActionChains(driver).click(username_field).send_keys("sawaco")\
    #                               .click(password_field).send_keys("123456")\
    #                               .send_keys(Keys.RETURN)
    # actions.perform()

    # METHOD 3
    # --------
    # wait = WebDriverWait(driver, 10)
    # username = wait.until(EC.element_to_be_clickable(
    #     (By.XPATH, "//input[@name='ctl00$ContentPlaceHolder1$ucLogin1$Login1$UserName']")))
    # username.send_keys("sawaco")
    # password = wait.until(EC.element_to_be_clickable(
    #     (By.XPATH, "//input[@name='ctl00$ContentPlaceHolder1$ucLogin1$Login1$Password']")))
    # password.send_keys("123456")
    # login_btn = driver.find_element_by_xpath("//input[@id='ContentPlaceHolder1_ucLogin1_Login1_LoginButton']")
    # login_btn.click()

    wait.until(EC.presence_of_all_elements_located((By.ID, "RAD_SLIDING_PANE_TEXT_RadSlidingPane2")))
    driver.maximize_window()


def get_locations_list(driver):
    """Return the list of all locations/meters available in the dropdown.
    """
    elem = rewind_dropdown_list(driver, 1)
    locations = []
    while True:
        loc = get_highlighted_location(driver)
        if locations and loc == locations[-1]:
            break
        locations.append(loc)
        elem.send_keys(Keys.ARROW_DOWN + Keys.ENTER)
    rewind_dropdown_list(elem, len(locations))
    return locations


def get_highlighted_location(driver):
    """Read the name of the location/meter that are
    currently highlighted in the dropdown list.

    Example:
    value = {"logEntries":[],"value":"Vuon uom Tan Thuan (ÐC)","text":"Vuon uom Tan Thuan (ĐC)","enabled":true,"checkedIndices":[],"checkedItemsTextOverflows":false}
    """
    value = driver.find_element_by_xpath(
        "//input[@id='ctl00_ContentPlaceHolder1_ucDailyReportConsumer_cboSites_ClientState']") \
        .get_attribute("value")
    val = value.split(',')[1].split(':')[1].strip('"')
    text = value.split(',')[2].split(':')[1].strip('"')
    return {'value': val, 'text': text}


def rewind_dropdown_list(driver, num_steps):
    """Go to the beginning of the dropdown list.
    """
    element = driver.find_element_by_xpath(
        "//input[@id='ctl00_ContentPlaceHolder1_ucDailyReportConsumer_cboSites_Input']")
    element.click()
    element.send_keys(Keys.ARROW_DOWN)
    for i in range(num_steps):
        element.send_keys(Keys.ARROW_UP)
    element.send_keys(Keys.ENTER)
    return element


def select_a_location(driver, list_length, location):
    """Select a location/meter from the dropdown list.
    """
    elem = rewind_dropdown_list(driver, list_length)
    elem.click()

    previous_loc = None
    while True:
        current_loc = get_highlighted_location(driver)
        # print(current_loc)
        if current_loc == previous_loc:
            print("Couldn't find the requested location/meter.")
            return False
        if location in current_loc.values():
            return True
        previous_loc = current_loc
        elem.send_keys(Keys.ARROW_DOWN + Keys.ENTER)


class TableWait(object):
    def __init__(self, driver, timeout=10):
        self.driver = driver
        self.timeout = timeout

    def __enter__(self):
        tables = self.driver.find_elements_by_xpath("//table[@id='ctl00_ContentPlaceHolder1_ucDailyReportConsumer_grv_ctl00']")
        if len(tables) == 0:
            self.begin = True
        else:
            self.begin = False
            self.old_table = tables[0]

        # Click on the Xem button
        self.driver.find_element_by_xpath(
            "//input[@id='ctl00_ContentPlaceHolder1_ucDailyReportConsumer_btnView_input']").click()

    def __exit__(self, *_):
        if self.begin:
            # WebDriverWait(self.driver, self.timeout).until(
            #     EC.element_to_be_clickable((By.ID, "ctl00_ContentPlaceHolder1_ucDailyReportConsumer_grv_ctl00")))
            wait_for(self.table_has_shown, self.timeout)
        else:
            wait_for(self.table_has_updated, self.timeout)

    def table_has_shown(self):
        tables = self.driver.find_elements_by_xpath("//table[@id='ctl00_ContentPlaceHolder1_ucDailyReportConsumer_grv_ctl00']")
        return len(tables) > 0

    def table_has_updated(self):
        new_table = self.driver.find_element_by_xpath("//table[@id='ctl00_ContentPlaceHolder1_ucDailyReportConsumer_grv_ctl00']")
        return new_table.id != self.old_table.id


def wait_for(condition_function, timeout):
    start_time = time.time()
    while time.time() < start_time + timeout:
        if condition_function():
            print("Waited successfully in {:.1f} sec.".format(time.time() - start_time))
            return True
        else:
            time.sleep(0.1)
    raise Exception("Timeout {} sec waiting for {}.".format(timeout, condition_function.__name__))
    # print("  Timeout waiting for table update.")
    # return False



In [2]:
# Use PhantomJS to browse the page
caps = webdriver.DesiredCapabilities.PHANTOMJS
caps['phantomjs.page.settings.userAgent'] = USER_AGENT
driver = webdriver.PhantomJS(executable_path=PHANTOMJS_EXE, desired_capabilities=caps)

driver.get(URL_LOGIN)
site_log_in(driver, ACCOUNT)

# Now that you are logged in, go to the page of interest
driver.get(URL_LOGGER)
wait = WebDriverWait(driver, 10)
wait.until(EC.element_to_be_clickable((By.ID, "ctl00_ContentPlaceHolder1_ucDailyReportConsumer_btnView_input")))

element = driver.find_element_by_xpath("//input[@id='ctl00_ContentPlaceHolder1_ucDailyReportConsumer_dtmStart_dateInput']")
element.clear()
element.send_keys(START_DATE)

element = driver.find_element_by_xpath("//input[@id='ctl00_ContentPlaceHolder1_ucDailyReportConsumer_dtmEnd_dateInput']")
element.clear()
element.send_keys(END_DATE)

locations = get_locations_list(driver)
# for i in locations:
#     print(i)


In [3]:
for i in locations:
    print(i)

{'value': '1 DUONG 15', 'text': '1 DUONG 15'}
{'value': '102 LE QUOC HUNG', 'text': '102 LE QUOC HUNG'}
{'value': '1041 TRAN XUAN SOAN', 'text': '1041 TRAN XUAN SOAN'}
{'value': '109 BEN VAN DON', 'text': '109 BEN VAN DON'}
{'value': '1243 LE VAN LUONG', 'text': '1243 LE VAN LUONG'}
{'value': '131-133 KHANH HOI-DOMINO', 'text': '131-133 KHANH HOI-DOMINO'}
{'value': '1368 LE VAN LUONG', 'text': '1368 LE VAN LUONG'}
{'value': '1419 LE VAN LUONG', 'text': '1419 LE VAN LUONG'}
{'value': '148 LAM VAN BEN', 'text': '148 LAM VAN BEN'}
{'value': '148 TON DAN', 'text': '148 TON DAN'}
{'value': '15A PHU LONG', 'text': '15A PHU LONG'}
{'value': '16 DUONG CAT LOI', 'text': '16 DUONG CAT LOI'}
{'value': '16/8C BUI VAN BA', 'text': '16/8C BUI VAN BA'}
{'value': '1716 HUYNH TAN PHAT', 'text': '1716 HUYNH TAN PHAT'}
{'value': '176 NGUYEN THI THAP', 'text': '176 NGUYEN THI THAP'}
{'value': '183C TON THAT THUYET', 'text': '183C TON THAT THUYET'}
{'value': '183F TON THAT THUYET', 'text': '183F TON THAT T

In [4]:
for i, loc in enumerate(XNTD):
    print("\nLocating '{}' ...".format(loc))
    if loc not in [item['value'] for item in locations] and \
            loc not in [item['text'] for item in locations]:
        print("** '{}' not found.".format(loc))
        continue
    if select_a_location(driver, len(locations), loc):
        print("OK.")
        table_wait = TableWait(driver, timeout=30)
        try:
            with table_wait:
                pass
        except Exception as msg:
            print("** Omitting '{}': {}".format(str(loc), msg))
            continue
        driver.save_screenshot('screenshoot_{}_{}.png'.format(i, loc))



Locating '102 LE QUOC HUNG' ...
OK.
Waited successfully in 2.6 sec.


In [5]:
# scrape_table_per_location(driver)

element = driver.find_element_by_xpath(
    "//*[@id='ctl00_ContentPlaceHolder1_ucDailyReportConsumer_grv_ctl00__0']/td[2]")
print("  --> " + element.text)


# driver.close()

  --> 102 LE QUOC HUNG


In [64]:
from bs4 import BeautifulSoup

print("Inside of scrape_table_per_location()")
tbody_elem = driver.find_element_by_xpath("//table[@id='ctl00_ContentPlaceHolder1_ucDailyReportConsumer_grv_ctl00']/tbody")
tbody_src = tbody_elem.get_attribute('innerHTML')
soup = BeautifulSoup(tbody_src, 'html.parser')


Inside of scrape_table_per_location()


In [20]:
rows = soup.find_all('tr')
print(rows)

[<tr class="rgRow" id="ctl00_ContentPlaceHolder1_ucDailyReportConsumer_grv_ctl00__0" style="text-align:center;">
<td>01/09/2017</td><td align="left">102 LE QUOC HUNG</td><td>
<table cellpadding="0" cellspacing="0" width="100%">
<tbody><tr>
<td>
                                                        41.10
                                                    </td>
<td>
                                                        30.96
                                                    </td>
<td>
                                                        16.20
                                                    </td>
</tr>
</tbody></table>
</td><td>743.10</td><td>
<table cellpadding="0" cellspacing="0" width="100%">
<tbody><tr>
<td>
                                                        23.40
                                                    </td>
<td>
                                                        17.60
                                                    </td>
<td>
                 

In [65]:
data = []
for row in soup.find_all('tr'):
    row_data = [td.get_text() for td in row.find_all('td')]
    row_data = [t.replace('\n', '').strip() for t in row_data]  # stripping
#     row_data = [row_data[i-1] for i in (1, 2, 4, 5, 6, 7, 9, 10, 11)]  # columns of interest, 1-base, from R
    data.append(row_data)
print(data[0])

['01/09/2017', '102 LE QUOC HUNG', '41.10                                                                                                            30.96                                                                                                            16.20', '41.10', '30.96', '16.20', '743.10', '23.40                                                                                                            17.60                                                                                                            13.90', '23.40', '17.60', '13.90', 'Đồ thị                                            |                                            Bảng']


In [76]:
a = data[6]
a

['03/09/2017',
 '102 LE QUOC HUNG',
 '43.80                                                                                                            31.73                                                                                                            14.30',
 '43.80',
 '31.73',
 '14.30',
 '761.60',
 '25.00                                                                                                            19.61                                                                                                            16.40',
 '25.00',
 '19.61',
 '16.40',
 'Đồ thị                                            |                                            Bảng']

In [77]:
[a[i-1] for i in (1, 2, 4, 5, 6, 7, 9, 10, 11)]

['03/09/2017',
 '102 LE QUOC HUNG',
 '43.80',
 '31.73',
 '14.30',
 '761.60',
 '25.00',
 '19.61',
 '16.40']

In [83]:
soup2 = BeautifulSoup(driver.page_source, 'html.parser')
table = soup2.find("table", attrs={"class": "rgMasterTable"})
# table = table.find_all("tbody")
print(len(table))
data = []
for row in table.find_all('tr'):
    row_data = [td.get_text().replace('\n', '').strip() for td in row.find_all('td')]
    data.append(row_data)
data[0]
data[1]
# data[7]

7


['Lưu lượng (m3/h)']

In [84]:
retained_columns = (1, 2, 4, 5, 6, 7, 9, 10, 11) 

In [85]:
max(retained_columns)

11

In [86]:
retained_columns - 1


TypeError: unsupported operand type(s) for -: 'tuple' and 'int'

In [132]:
tbody_elem = driver.find_element_by_xpath("//table[@id='ctl00_ContentPlaceHolder1_ucDailyReportConsumer_grv_ctl00']/tbody")
tbody_src = tbody_elem.get_attribute('innerHTML')
soup = BeautifulSoup(tbody_src, 'html.parser')

retained_columns = (0, 1, 3, 4, 5, 6, 8, 9, 10)  # columns of interest, from R
data = []
for row in soup.find_all('tr'):
    row_data = [td.get_text() for td in row.find_all('td')]
    if len(row_data) < len(retained_columns):
        continue
    row_data = [t.replace('\n', '').strip() for t in row_data]  # stripping
    row_data = [row_data[i] for i in retained_columns]
    data.append(row_data)
print(data)

[['01/09/2017', '102 LE QUOC HUNG', '41.10', '30.96', '16.20', '743.10', '23.40', '17.60', '13.90'], ['02/09/2017', '102 LE QUOC HUNG', '40.80', '31.91', '17.90', '765.90', '24.60', '18.29', '15.10'], ['03/09/2017', '102 LE QUOC HUNG', '43.80', '31.73', '14.30', '761.60', '25.00', '19.61', '16.40'], ['04/09/2017', '102 LE QUOC HUNG', '44.20', '31.52', '15.30', '756.40', '24.60', '19.22', '15.70'], ['05/09/2017', '102 LE QUOC HUNG', '40.30', '31.72', '15.10', '761.20', '23.40', '17.65', '14.20']]


In [133]:
len(data)

5

In [135]:
import pandas as pd
headers = ["DATE", "LOCATION", "FLOW_MAX", "FLOW_AVG", "FLOW_MIN",
           "VOLUME", "PRESS_MAX", "PRESS_AVG", "PRESS_MIN"]

In [137]:
df = pd.DataFrame(data, columns=headers)
df

,DATE,LOCATION,FLOW_MAX,FLOW_AVG,FLOW_MIN,VOLUME,PRESS_MAX,PRESS_AVG,PRESS_MIN
0,01/09/2017,102 LE QUOC HUNG,41.10,30.96,16.20,743.10,23.40,17.60,13.90
1,02/09/2017,102 LE QUOC HUNG,40.80,31.91,17.90,765.90,24.60,18.29,15.10
2,03/09/2017,102 LE QUOC HUNG,43.80,31.73,14.30,761.60,25.00,19.61,16.40
3,04/09/2017,102 LE QUOC HUNG,44.20,31.52,15.30,756.40,24.60,19.22,15.70
4,05/09/2017,102 LE QUOC HUNG,40.30,31.72,15.10,761.20,23.40,17.65,14.20


In [188]:
df[0:2]

,DATE,LOCATION,FLOW_MAX,FLOW_AVG,FLOW_MIN,VOLUME,PRESS_MAX,PRESS_AVG,PRESS_MIN
0,01/09/2017,102 LE QUOC HUNG,41.10,30.96,16.20,743.10,23.40,17.60,13.90
1,02/09/2017,102 LE QUOC HUNG,40.80,31.91,17.90,765.90,24.60,18.29,15.10


In [189]:
df[['LOCATION', 'VOLUME']]

,LOCATION,VOLUME
0,102 LE QUOC HUNG,743.10
1,102 LE QUOC HUNG,765.90
2,102 LE QUOC HUNG,761.60
3,102 LE QUOC HUNG,756.40
4,102 LE QUOC HUNG,761.20


In [435]:
soup = BeautifulSoup(driver.page_source, 'html.parser')
soup = soup.find("table", attrs={"class": "rgMasterTable"})

retained_columns = (0, 1, 3, 4, 5, 6, 8, 9, 10)  # columns of interest, from R
data = []
for tr in soup.find_all('tr'):
    row = [td.get_text() for td in tr.find_all('td')]
    if len(row) < len(retained_columns):
        continue
    row = [' '.join(d.split()) for d in row]  # stripping
    row = [row[i] for i in retained_columns]
    data.append(row)
print(data)
print(len(data))

import pandas as pd
headers = ["DATE", "LOCATION", "FLOW_MAX", "FLOW_AVG", "FLOW_MIN",
           "VOLUME", "PRESS_MAX", "PRESS_AVG", "PRESS_MIN"]
# headers = ["date", "location", "flow_max", "flow_avg", "flow_min",
#            "volume", "press_max", "press_avg", "press_min"]
df = pd.DataFrame(data, columns=headers)
df

[['01/09/2017', '102 LE QUOC HUNG', '41.10', '30.96', '16.20', '743.10', '23.40', '17.60', '13.90'], ['02/09/2017', '102 LE QUOC HUNG', '40.80', '31.91', '17.90', '765.90', '24.60', '18.29', '15.10'], ['03/09/2017', '102 LE QUOC HUNG', '43.80', '31.73', '14.30', '761.60', '25.00', '19.61', '16.40'], ['04/09/2017', '102 LE QUOC HUNG', '44.20', '31.52', '15.30', '756.40', '24.60', '19.22', '15.70'], ['05/09/2017', '102 LE QUOC HUNG', '40.30', '31.72', '15.10', '761.20', '23.40', '17.65', '14.20']]
5


,DATE,LOCATION,FLOW_MAX,FLOW_AVG,FLOW_MIN,VOLUME,PRESS_MAX,PRESS_AVG,PRESS_MIN
0,01/09/2017,102 LE QUOC HUNG,41.10,30.96,16.20,743.10,23.40,17.60,13.90
1,02/09/2017,102 LE QUOC HUNG,40.80,31.91,17.90,765.90,24.60,18.29,15.10
2,03/09/2017,102 LE QUOC HUNG,43.80,31.73,14.30,761.60,25.00,19.61,16.40
3,04/09/2017,102 LE QUOC HUNG,44.20,31.52,15.30,756.40,24.60,19.22,15.70
4,05/09/2017,102 LE QUOC HUNG,40.30,31.72,15.10,761.20,23.40,17.65,14.20


In [463]:
df2 = df.copy()
df2["DATE"] = pd.to_datetime(df2["DATE"], dayfirst=True).dt.date
df2 = df2.set_index('DATE')

list(map(lambda x: x.strftime("%Y-%m-%d"), [df2.index.min(), df2.index.max()]))

# df2.index.min().strftime("%Y-%m-%d")
df2.index[0].strftime("%Y-%m-%d")
df2.index[-1].strftime("%Y-%m-%d")


filename = "daily__{}__{}__{}.csv".format(loc,
                                          df2.index.min().strftime("%Y-%m-%d"),
                                          df2.index.max().strftime("%Y-%m-%d"))

# df3 = pd.read_csv(filename, index_col='DATE')

df3 = pd.read_csv(filename)
df3["DATE"] = pd.to_datetime(df3["DATE"], dayfirst=True).dt.date
df3 = df3.set_index('DATE')

df3.index


Index([2017-09-01, 2017-09-02, 2017-09-03, 2017-09-04, 2017-09-05], dtype='object', name='DATE')

In [506]:
import sys
sys.getfilesystemencoding()

import unicodedata
def remove_accents(input_str):
#     input_str = input_str.decode('windows-1252')
    nkfd_form = unicodedata.normalize('NFKD', input_str)
    output_str = "".join([c for c in nkfd_form if not unicodedata.combining(c)])
    return str(output_str)

remove_accents(filename)
b = unicodedata.normalize('NFKD', filename).encode('ASCII', 'ignore')
b.decode()

# ord('Đ')
# ord('Ð')
# ord('Đ')

'daily__Vuon uom Tan Thuan (C)__2017-09-01__2017-09-05.csv'

In [500]:
# filename = "daily__Vuon uom Tan Thuan (ĐC)__2017-09-01__2017-09-05.csv"
# f = open(filename)
# pd.read_csv(filename, index_col='date')
f.close()

In [411]:
df.DATE = pd.to_datetime(df.DATE, dayfirst=True)
df["DATE"] = df["DATE"].dt.date
print(df.DATE)
max(df["DATE"]).strftime("%Y-%m-%d")
# df.DATE.apply(lambda x: x.strftime("%Y-%m-%d"))

# print(df.DATE)

# df.DATE.dt.strftime("%Y-%m-%d")
# df.DATE = df.DATE.dt.date
# df.DATE.str

# df.iloc[:, 0]

0    2017-09-01
1    2017-09-02
2    2017-09-03
3    2017-09-04
4    2017-09-05
Name: DATE, dtype: object


'2017-09-05'

In [338]:
df = df.set_index('DATE')

In [339]:
df

,LOCATION,FLOW_MAX,FLOW_AVG,FLOW_MIN,VOLUME,PRESS_MAX,PRESS_AVG,PRESS_MIN
DATE,,,,,,,,
01/09/2017,102 LE QUOC HUNG,41.10,30.96,16.20,743.10,23.40,17.60,13.90
02/09/2017,102 LE QUOC HUNG,40.80,31.91,17.90,765.90,24.60,18.29,15.10
03/09/2017,102 LE QUOC HUNG,43.80,31.73,14.30,761.60,25.00,19.61,16.40
04/09/2017,102 LE QUOC HUNG,44.20,31.52,15.30,756.40,24.60,19.22,15.70
05/09/2017,102 LE QUOC HUNG,40.30,31.72,15.10,761.20,23.40,17.65,14.20


In [340]:
df.iloc[0, 1]

'41.10'

In [341]:
df.index

Index(['01/09/2017', '02/09/2017', '03/09/2017', '04/09/2017', '05/09/2017'], dtype='object', name='DATE')

In [336]:
df.index = pd.to_datetime(df.index, dayfirst=True)
print(df.index)
df.loc['20170902', 'FLOW_MAX']
df.index[0]
df.index.dt.date

DatetimeIndex(['2017-09-01', '2017-09-02', '2017-09-03', '2017-09-04',
               '2017-09-05'],
              dtype='datetime64[ns]', name='DATE', freq=None)


AttributeError: 'DatetimeIndex' object has no attribute 'dt'

In [322]:
df.to_csv(loc + ".csv")

In [327]:
df2 = pd.read_csv(loc + ".csv", index_col='DATE')

In [328]:
df2

,LOCATION,FLOW_MAX,FLOW_AVG,FLOW_MIN,VOLUME,PRESS_MAX,PRESS_AVG,PRESS_MIN
DATE,,,,,,,,
2017-09-01,102 LE QUOC HUNG,41.1,30.96,16.2,743.1,23.4,17.60,13.9
2017-09-02,102 LE QUOC HUNG,40.8,31.91,17.9,765.9,24.6,18.29,15.1
2017-09-03,102 LE QUOC HUNG,43.8,31.73,14.3,761.6,25.0,19.61,16.4
2017-09-04,102 LE QUOC HUNG,44.2,31.52,15.3,756.4,24.6,19.22,15.7
2017-09-05,102 LE QUOC HUNG,40.3,31.72,15.1,761.2,23.4,17.65,14.2


In [333]:
df2.iloc[1, 3]
df2['FLOW_MAX']
# dates = pd.date_range('1/1/2000', periods=8)
# dates
# dates[1]
# pd.to_datetime(df2['DATE'], dayfirst=True)
print(df2.index)

Index(['2017-09-01', '2017-09-02', '2017-09-03', '2017-09-04', '2017-09-05'], dtype='object', name='DATE')


In [231]:
data = [['Australia', 100], ['France', 200], ['Germany', 300], ['America', 400]]
df = pd.DataFrame(data, columns=['Country', 'Volume']).set_index('Country')
df

,Volume
Country,
Australia,100
France,200
Germany,300
America,400


In [232]:
print(df)

           Volume
Country          
Australia     100
France        200
Germany       300
America       400


In [233]:
df['Volume']['France']

200

In [234]:
L = ['Thanks You', 'Its fine no problem', 'Are you sure']
df = pd.DataFrame(L)

In [235]:
print(df)

                     0
0           Thanks You
1  Its fine no problem
2         Are you sure


In [236]:
df = pd.DataFrame({'col': L})
print(df)

                   col
0           Thanks You
1  Its fine no problem
2         Are you sure


In [237]:
df = pd.DataFrame({'col': [1, 2, 3]})
df['new_col'] = L
print (df)

   col              new_col
0    1           Thanks You
1    2  Its fine no problem
2    3         Are you sure


In [238]:
pd.DataFrame({'col1': [1, 2, 3, 4], 'col2': L})

ValueError: arrays must all be same length

In [ ]:
data = [['Name','Rank','Complete'],
               ['one', 1, 1],
               ['two', 2, 1],
               ['three', 3, 1],
               ['four', 4, 1],
               ['five', 5, 1]]
df = pd.DataFrame(data)
print(df)

In [ ]:
headers = data.pop(0)
df = pd.DataFrame(data, columns=headers).set_index(headers[0])
print(df)

In [ ]:
df = pd.DataFrame(data[1:], columns=data[0]).set_index(data[0][0])
print(df)

In [ ]:
d = {'col1': list(range(1,10)),
     'col2': list(range(11,20)),
     'col3': list(range(51,60))}
d

In [ ]:
d['col1']

In [ ]:
d['col3']

In [ ]:
len(d)

In [ ]:
d.keys()

In [ ]:
d.values()

In [ ]:
[len(v) for v in d.values()]

In [ ]:
list(d.values())[1]

In [ ]:
[d[col][2] for col in d.keys()]

In [ ]:
[d[col][2] for col in d]

In [ ]:
[col for col in d]

In [ ]:
{c: d[c][2] for c in d}

In [ ]:
list_of_dicts = [{c: d[c][i] for c in d} for i in range(len(d['col1']))]
list_of_dicts

In [ ]:
pd.DataFrame(list_of_dicts)

In [ ]:
d

In [ ]:
pd.DataFrame(d)

In [ ]:
pd.DataFrame(list_of_dicts) == pd.DataFrame(d)

In [ ]:
d

In [ ]:
list_of_dicts

In [ ]:
{key: [item[key] for item in list_of_dicts] for key in list_of_dicts[0]}